# Consumer Segmentation

Using clustering to segment consumers based on their 

consumption patterns and demographics. 
(Understand behaviors of specific consumer groups).

Features used for clustering:

energy consumption
monthly_expense
number_of_family_members
floor_area_of_the_household
usage_of_solar_panels
energy_source_used_to_cook

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('/lirneasia/data/lacuna/processed/sm_april_2024_dump/final_processed.csv')
survey_data = pd.read_csv('/lirneasia/data/lacuna/raw/Survey_data/Survey_wave_1_may_29.csv', low_memory=False)

ImportError: /home/manifold/miniconda3/lib/python3.11/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /home/manifold/miniconda3/lib/python3.11/site-packages/scipy/optimize/_highs/_highs_wrapper.cpython-311-x86_64-linux-gnu.so)